In [26]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


ERROR: Operation cancelled by user


In [27]:
# from huggingface_hub import notebook_login
# notebook_login()

In [28]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
label2id = {'Corrective':0,'Adaptive':1,'Perfective':2}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
# train = train.rename(columns={'3_labels':'label','comment':'text'})
train = train.replace({"label": label2id})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
# test = test.rename(columns={'3_labels':'label','comment':'text'})
test = test.replace({"label": label2id})
test.fillna(0, inplace=True)
print(len(test))



1758


In [29]:
# from datasets import Dataset, load_metric

In [30]:
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [31]:
len(train)

35

In [32]:
train['label'].value_counts()

2    18
0    11
1     6
Name: label, dtype: int64

In [33]:
test['label'].value_counts()

0    592
1    584
2    582
Name: label, dtype: int64

In [34]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [35]:
train['embeddings'] = train['text'].apply(model.encode)
test['embeddings'] = test['text'].apply(model.encode)

In [36]:
test

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder,embeddings
1239,c0bae1256c20ddb0c76da865f2c0470fb0b1db1c,Mylyn Reviews,"322734: Avoid exception, when loading non-exis...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.049276624, 0.119949915, 0.0074695917, 0.13..."
903,4a29e164a8ca222fd8b0d2043e1d44494e84544e,spring-framework,Allow binary messages in StompSubProtocolHandl...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.02142287, 0.14910096, 0.08537317, -0.01865..."
1483,a0a251e62e4abb8f0660e96c654c120506e54bd9,Vala,gtk+-2.0: expose Gtk.CellEditable.start_editing\n,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.040489707, 0.07557194, -0.024681384, -0.04..."
1080,c22df00d95d89feed488608ad943822ed6a9c715,Vala,gio-2.0: Update. New Notification and Subproce...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.008856162, 0.18510188, 0.013986745, 0.0162..."
375,9cea634f7d4276b87821f3ab7c160fa67b2c85b1,hadoop,HDFS-2414. Fix TestDFSRollback to avoid spurio...,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.10534891, 0.078534015, 0.0031895277, 0.019..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,59f1ae7b2b15776314059123e26dc1563ca064c8,Vala,tracker-indexer-module-1.0: regenerate\n,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.06868401, 0.23774986, -0.07392164, 0.06734..."
319,3e714ddd9f062e534c1ee028ac8b2269a062004b,intellij-community,IDEADEV-11483 IE conditional comments support-...,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,"[-0.081156306, 0.029742366, -0.12290736, 0.072..."
1597,b2cc0ae3027251df10d4306358cb53fe314ea852,belaban$jgroups,- Replace Math.random() with ThreadLocalRandom...,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,"[-0.10087136, 0.0065665375, -0.024835767, 0.06..."
1109,c8dec345b6e2ad04ffcede24779dd75efd25d599,Vala,Add used DBus attribute and fix UnixMountEntry...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.19117478, -0.05123747, -0.04218266, 0.0345..."


In [37]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [38]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [39]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.5142207053469852
Logistic Regression Precision: 0.566839691914978
Logistic Regression Recall: 0.5142207053469852
Logistic Regression Recall: 0.4619635914550532
